In [1]:
import torch
import torch.utils.data as torch_split
import numpy as np
import dataset
import test
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from monai.losses import DiceLoss
from monai.networks.nets import UNet
from monai.data import DataLoader


In [2]:
path = "H:/Projects/Kaggle/CZII-CryoET-Object-Identification/datasets/3D/dim96-no-corner"
# data = dataset.UNetDataset(path=path)

# tv_split = 0.8
# trn = int(len(data) * tv_split)
# val = len(data) - trn

# train_dataset, val_dataset = torch_split.random_split(data, [trn, val])
train_dataset = dataset.UNetDataset(path=path, train=True)
val_dataset = dataset.UNetDataset(path=path, val=True)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model initialization
model = UNet(
    spatial_dims=3,
    in_channels=1,  # Assuming single-channel input (adjust as needed)
    out_channels=7,  # Assuming 7 classes for segmentation (adjust as needed)
    channels=(64, 128, 256, 512),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(device)

# model = test.UNet3D(in_channels = 1, out_channels = 7)

# Loss function and optimizer
criterion = DiceLoss(to_onehot_y=True, softmax=True).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=dataset.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=dataset.collate_fn)

n_batch = 5

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    n = 0
    c_loss = 0
    for batch in train_loader:
        n += 1
        inputs, targets = batch['src'].float().to(device), batch['tgt'].long().to(device)
        # print(inputs.shape)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # c_loss += loss.item()
        # if n % n_batch == 0:
        #     print(f"batch {n-n_batch}-{n} train loss: {c_loss}")
        #     c_loss = 0
        
        train_loss += loss.item()

    # Validation loop
    model.eval()
    val_loss = 0
    n = 0
    with torch.no_grad():
        for batch in val_loader:
            n += 1
            inputs, targets = batch['src'].float().to(device), batch['tgt'].long().to(device)

            outputs = model(inputs)

            loss = criterion(outputs, targets)

            # print(f"validation batch {n} done")
            
            val_loss += loss.item()

    # Print loss for this epoch
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

c:\Users\hmhor\AppData\Local\Programs\Python\Python310\lib\site-packages\monai\networks\nets\unet.py:130: UserWarning: `len(strides) > len(channels) - 1`, the last 1 values of strides will not be used.
  warnings.warn(f"`len(strides) > len(channels) - 1`, the last {delta} values of strides will not be used.")


Epoch 1/20 - Train Loss: 0.9238, Val Loss: 0.9140
Epoch 2/20 - Train Loss: 0.8955, Val Loss: 0.8938
Epoch 3/20 - Train Loss: 0.8726, Val Loss: 0.8755


KeyboardInterrupt: 